In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
# Use the correct relative path to the Data folder
data = pd.read_csv("../Data/heart_2020_cleaned.csv")

# Encode all object (string) columns automatically
le = LabelEncoder()
for col in data.select_dtypes(include=['object']).columns:
    data[col] = le.fit_transform(data[col])

# Drop rows with missing values if any
data = data.dropna()

data.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,1,0,0,3,30,0,0,55-59,White,2,1,4,5,1,0,1
1,No,20.34,0,0,1,0,0,0,0,80 or older,White,0,1,4,7,0,0,0
2,No,26.58,1,0,0,20,30,0,1,65-69,White,2,1,1,8,1,0,0
3,No,24.21,0,0,0,0,0,0,0,75-79,White,0,0,2,6,0,0,1
4,No,23.71,0,0,0,28,0,1,0,40-44,White,0,1,4,8,0,0,0


In [ ]:
# Features & Target
X = data.drop('HeartDisease', axis=1)
y = data['HeartDisease']  # Already encoded as 0/1

# Ensure all features are numeric (for safety)
if not all([pd.api.types.is_numeric_dtype(X[col]) for col in X.columns]):
    print("Non-numeric columns detected. Converting...")
    for col in X.columns:
        if not pd.api.types.is_numeric_dtype(X[col]):
            X[col] = pd.factorize(X[col])[0]

# Print dtypes for debugging
print("Feature dtypes:\n", X.dtypes)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Non-numeric columns detected. Converting...
Feature dtypes:
 BMI                 float64
Smoking               int64
AlcoholDrinking       int64
Stroke                int64
PhysicalHealth        int64
MentalHealth          int64
DiffWalking           int64
Sex                   int64
AgeCategory           int64
Race                  int64
Diabetic              int64
PhysicalActivity      int64
GenHealth             int64
SleepTime             int64
Asthma                int64
KidneyDisease         int64
SkinCancer            int64
dtype: object
